In [1]:
from regelum import Optimizable
import torch as th
from torch.optim import Adam
import casadi as cs
import numpy as np
from regelum.optimizable.core.configs import torch_default_config, casadi_default_config
from regelum.__utilities import rc

Torch optimization

In [2]:
def objective_torch(x, y):
    return (x**2 + y**2).sum()


x = th.Tensor([1.0, 2.0]).requires_grad_(True)
opt_torch = Adam(params=[x], lr=0.1)

for _ in range(5000):
    opt_torch.zero_grad()
    obj = objective_torch(th.Tensor([1.0, 2.0]), x)
    obj.backward()
    opt_torch.step()

x

tensor([-2.5223e-44,  0.0000e+00], requires_grad=True)

Casadi optimization

In [3]:
def objective_casadi(x, y):
    return cs.sumsqr(x) + cs.sumsqr(y)


opti = cs.Opti()
x = opti.parameter(2)
y = opti.variable(2)
objective_symb = objective_casadi(x, y)
opti.minimize(objective_symb)
opti.solver(
    "ipopt",
    {"print_in": False, "print_out": False, "print_time": True},
    {"print_level": 0},
)
opt_func = opti.to_function(
    "min_fun", [x], [y, objective_symb], ["x"], ["y", "objective_casadi"]
)
opt_func(cs.DM([1.0, 2.0]))


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  16.00us ( 16.00us) 952.00ns (952.00ns)         1
  nlp_grad_f  |  87.00us ( 43.50us)   5.20us (  2.60us)         2
       total  |  11.88ms ( 11.88ms) 739.47us (739.47us)         1


(DM([0, 0]), DM(5))

Regelum optimization

In [4]:
def universal_objective(x, y):
    return rc.sum(x**2 + (y - 1) ** 2)

In [5]:
opt_regelum = Optimizable(casadi_default_config)
x = opt_regelum.create_variable(2, 1, name="x", is_constant=True)
y = opt_regelum.create_variable(2, 1, name="y")
opt_regelum.register_objective(universal_objective, [x, y])
opt_regelum.optimize(x=cs.DM([1.0, 2.0]))

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  65.00us ( 32.50us)   3.87us (  1.93us)         2
  nlp_grad_f  | 110.00us ( 36.67us)   6.74us (  2.25us)         3
  nlp_hess_l  |  28.00us ( 28.00us)   1.63us (  1.63us)         1
       total  |  12.66ms ( 12.66ms) 790.92us (790.92us)         1


{'y': DM([1, 1])}

In [6]:
torch_default_config.config_options["n_epochs"] = 10000
opt_regelum = Optimizable(torch_default_config)
x = opt_regelum.create_variable(2, 1, name="x", is_constant=True)
y = opt_regelum.create_variable(
    2, 1, name="y", like=th.Tensor([1.0, 2.0]).requires_grad_(True)
)
opt_regelum.register_objective(universal_objective, [x, y])
opt_regelum.optimize(x=th.FloatTensor([1.0, 2.0]))

VarContainer:
  y
  data: tensor([1.0000, 1.0000], requires_grad=True)
  metadata: tensor([1.0000, 1.0000], requires_grad=True)
  dims: (2, 1)
  is_constant: False


  

Working with models

In [7]:
from regelum.model import ModelWeightContainer, ModelWeightContainerTorch

In [8]:
model = ModelWeightContainer(2, cs.DM([[1.0, 2.0], [3.0, 4.0]]))
model("whatever")

DM([[1, 2]])

In [9]:
model_torch = ModelWeightContainerTorch(2)

In [10]:
model.weights, model.cache.weights

(DM(
 [[1, 2], 
  [3, 4]]),
 DM(
 [[1, 2], 
  [3, 4]]))

In [11]:
opt_regelum = Optimizable(casadi_default_config)
x = opt_regelum.create_variable(1, 2, name="x", is_constant=True)
w = opt_regelum.create_variable(name="w", like=model.named_parameters)
argin_to_model = opt_regelum.create_variable(
    1, 2, name="argi_to_model", is_constant=True
)
y = opt_regelum.create_variable(1, 2, name="y", is_nested_function=True)
opt_regelum.connect_source(connect_to=y, func=model, source=argin_to_model, weights=w)
opt_regelum.register_objective(universal_objective, [x, y])
opt_regelum.optimize(x=cs.DM([1.0, 2.0]), y=cs.DM([1.0, 2.0]))

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   6.00us (  2.00us)   5.67us (  1.89us)         3
  nlp_grad_f  |   8.00us (  2.00us)   7.54us (  1.88us)         4
  nlp_hess_l  |   3.00us (  1.50us)   2.67us (  1.34us)         2
       total  | 792.00us (792.00us) 790.91us (790.91us)         1


{'w': DM(
 [[1, 1], 
  [0, 0]])}

In [12]:
opt_regelum = Optimizable(torch_default_config)
x = opt_regelum.create_variable(1, 2, name="x", is_constant=True)
w = opt_regelum.create_variable(name="w", like=model_torch.named_parameters)
argin_to_model = opt_regelum.create_variable(
    1, 2, name="argi_to_model", is_constant=True, like=th.FloatTensor([1.0, 2.0])
)
y = opt_regelum.create_variable(1, 2, name="y", is_nested_function=True)
opt_regelum.connect_source(
    connect_to=y, func=model_torch, source=argin_to_model, weights=w
)
opt_regelum.register_objective(universal_objective, [x, y])
opt_regelum.optimize(x=th.FloatTensor([1.0, 2.0]))

VarContainer:
  w
  data: <bound method Module.named_parameters of ModelWeightContainerTorch()>
  metadata: <bound method Module.named_parameters of ModelWeightContainerTorch()>
  dims: ()
  is_constant: False


  y
  data: tensor([1.0000, 1.0000], grad_fn=<SliceBackward0>)
  metadata: tensor([1.0000, 1.0000], grad_fn=<SliceBackward0>)
  dims: (1, 2)
  is_constant: False


  

Inherit from `Optimizable`

In [13]:
from regelum.model import ModelPerceptron, ModelQuadLin

In [49]:
class MyOptimizableObject(Optimizable):
    def __init__(self, model, opt_config):
        super().__init__(opt_config)
        self.model = model
        self.initialize_optimization_procedure()

    def initialize_optimization_procedure(self):
        self.model_weights = self.create_variable(
            name="model_weights",
            like=self.model.named_parameters,
        )
        self.model_argin = self.create_variable(
            3, 2, name="model_argin", is_constant=True
        )
        self.model_output_reference = self.create_variable(
            3, 1, name="model_output_reference", is_constant=True
        )
        self.model_output = self.create_variable(
            name="model_output", is_nested_function=True
        )
        self.connect_source(
            connect_to=self.model_output,
            func=self.model,
            source=self.model_argin,
            weights=self.model_weights,
        )
        self.register_objective(
            self.objective_function, [self.model_output, self.model_output_reference]
        )

    def objective_function(self, model_output_reference, model_output):
        return rc.sum((model_output_reference - model_output) ** 2)

In [50]:
model = ModelPerceptron(2, 1, 3, 3)

In [51]:
torch_default_config.config_options["n_epochs"] = 10000
optbl = MyOptimizableObject(model, torch_default_config)

In [52]:
optbl.optimize(
    model_output_reference=th.FloatTensor([[1.0], [3], [5]]),
    model_argin=th.FloatTensor([[0.0, 0.0], [1, 2], [3, 5]]),
)

VarContainer:
  model_weights
  data: <bound method Module.named_parameters of ModelPerceptron(
  (input_layer): Linear(in_features=2, out_features=3, bias=True)
  (hidden_layers): ModuleList(
    (0-2): 3 x Linear(in_features=3, out_features=3, bias=True)
  )
  (output_layer): Linear(in_features=3, out_features=1, bias=True)
)>
  metadata: <bound method Module.named_parameters of ModelPerceptron(
  (input_layer): Linear(in_features=2, out_features=3, bias=True)
  (hidden_layers): ModuleList(
    (0-2): 3 x Linear(in_features=3, out_features=3, bias=True)
  )
  (output_layer): Linear(in_features=3, out_features=1, bias=True)
)>
  dims: ()
  is_constant: False


  model_output
  data: tensor([[1.],
        [3.],
        [5.]], grad_fn=<AddmmBackward0>)
  metadata: tensor([[1.],
        [3.],
        [5.]], grad_fn=<AddmmBackward0>)
  dims: ()
  is_constant: False


  

In [53]:
optbl.model_output()

tensor([[1.],
        [3.],
        [5.]], grad_fn=<AddmmBackward0>)

In [54]:
model = ModelQuadLin("symmetric", is_with_linear_terms=False, dim_inputs=2)

In [55]:
optbl = MyOptimizableObject(model, casadi_default_config)

In [56]:
new_weights = optbl.optimize(
    model_output_reference=cs.DM([[1.0], [3], [5]]),
    model_argin=cs.DM([[0.0, 0.0], [1, 2], [3, 5]]),
)["model_weights"]

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  23.00us (  5.75us)  20.75us (  5.19us)         4
  nlp_grad_f  |  27.00us (  5.40us)  26.77us (  5.35us)         5
  nlp_hess_l  |  21.00us (  7.00us)  21.18us (  7.06us)         3
       total  |   2.04ms (  2.04ms)   2.02ms (  2.02ms)         1


In [59]:
model(cs.DM([[0.0, 0.0], [1, 2], [3, 5]]), weights=new_weights)

DM([0, 3, 5])